## Задание 1 (8 баллов).
В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста. Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).

Подсказки:

- нужно будет добавить еще один тэг <start>  
- еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы  
- тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

In [1]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
import pandas as pd
from IPython.display import Image
from IPython.core.display import HTML 
from collections import Counter

In [2]:
news = open('lenta.txt', encoding='utf8').read()

In [3]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text



In [174]:
norm_news = normalize(news)
norm_news, norm_news_test = norm_news[:10000], norm_news[10000:]

In [175]:
vocab_news = Counter(norm_news)

In [176]:
from nltk.tokenize import sent_tokenize

def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

def words_to_ngrams(words, n = 3, sep=" "):
    return [sep.join(words[i:i+n]) for i in range(len(words)-n+1)]

In [184]:
sentences_news = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]

In [185]:
unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(words_to_ngrams(sentence))

In [186]:
bigrams_news.most_common(10)

[('<start> <start>', 76344),
 ('<start> в', 7972),
 ('<start> по', 6211),
 ('<start> как', 3738),
 ('риа новости', 3504),
 ('по словам', 1971),
 ('об этом', 1795),
 ('<start> однако', 1694),
 ('<start> на', 1643),
 ('что в', 1624)]

In [187]:
trigrams_news.most_common(10)

[('<start> <start> в', 7972),
 ('<start> <start> по', 6211),
 ('<start> <start> как', 3738),
 ('<start> <start> однако', 1694),
 ('<start> <start> на', 1643),
 ('<start> <start> об', 1619),
 ('<start> об этом', 1579),
 ('<start> <start> он', 1553),
 ('<start> по словам', 1549),
 ('сообщает риа новости', 1324)]

In [8]:
from scipy.sparse import csr_matrix, csc_matrix, lil_matrix

In [188]:
matrix_news = lil_matrix((len(bigrams_news), 
                   len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}

id2bigram_news = list(bigrams_news)
bigram2id_news = {word:i for i, word in enumerate(id2bigram_news)}

for ngram in trigrams_news:
    word1, word2, word3 = ngram.split(' ')
    bigram = word1 + ' ' + word2
    matrix_news[bigram2id_news[bigram], word2id_news[word3]] = (trigrams_news[ngram]/bigrams_news[bigram])

In [189]:
matrix_news = csr_matrix(matrix_news)

In [190]:
def generate(matrix, id2bigram, id2word, bigram2id, n=200, start='<start> <start>'):
    text = []
    current_idx = bigram2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(list(range(matrix.shape[1])), p=matrix[current_idx].toarray()[0])
        
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            current_idx = bigram2id[start]
        
        else:
            part = id2bigram[current_idx] + ' ' + id2word[chosen]
            part = ' '.join(part.split()[1:])
            current_idx = bigram2id[part]
    
    return ' '.join(text)

In [191]:
print(generate(matrix_news, id2bigram_news, id2word_news, bigram2id_news).replace('<end>', '\n'))

по данным командования части в которой in означает intelligence разведка it информационные технологии крайне важный аспект нашей стабильности развития нации а равно и петербургском метро вероятно будет третьим членом экипажа наряду с сша о предоставлении россии очередного кредитного транша по 50 миллионов долларов и специалисты мчс 
 одна из электростанций компании 
 мы практически завершим эту работу следует по его словам полевой командир хамзат батаев считавшийся командующим бамутским направлением сопротивления чеченских боевиков 
 по состоянию на 13.00 московского времени в воскресенье вечером в воскресенье 26 декабря конкурса красоты семь красавиц 
 таким образом мнение писателей представляет собой самую крупную с 1980-х годов бессрочную забастовку сообщает риа новости путин прибыл в псков в ближайшие полтора года рейтинг столичного градоначальника 
 в этом регионе и не меньше месяца 
 при задержании бабицкого 
 напомним что вчера президента не может быть приравнена к военным мерам

In [192]:
print(generate(matrix_news, id2bigram_news, id2word_news, bigram2id_news).replace('<end>', '\n'))

с другими членами руководства совета европы во главе с исполнительным секретарем снг юрием яровым сказал что на прошедшем в понедельник носит так как является налоговой тайной 
 по данным следствия ялькин установил программу отслеживающую ответы информационной системы таможенных органов 
 таким образом держать в напряжении республику 
 по мнению главы цик альбом существует в природе 
 что же касается потерь федеральных сил чьи интересы могут затрагивать зарегистрированные на острове сан мигель 
 по словам представителей фсб финансовые средства официально направляемые на социальные нужды 
 председатель испанского правительства хосе мариа аснаром 
 российские военные обнаружили наблюдательный пост боевиков 
 крупнейший американский провайдер aol активно рекламировал эту программу уверяя что она никогда его ни к чему не привели 
 все свидетельствовало о том что под его командованием 
 геннадий лузин был избран губернатором красноярского края проверяет причастность нескольких десятков стив

In [193]:
print(generate(matrix_news, id2bigram_news, id2word_news, bigram2id_news).replace('<end>', '\n'))

однако как полагают депутаты утвержденный правительством список товаров уже существенно сократился 
 об этом итар-тасс сообщили сегодня в гудермес грозный шали урус-мартан и другие нефтяные компании в районе села шатили сообщает риа новости администрация клинтона не будет 
 отвечая на вопрос зачем ты это сказал 
 площадь пожара достигала 200 квадратных метров на складе сдетонировала еще одна его встреча с верховным комиссаром оон по делам беженцев призвал российские власти чинить им не исполнится 19 лет от рака лимфатических сосудов который был подписан 
 об этом он заявил что огульные обвинения не делают чести ихавторам 
 для этого подготовительную работу 
 однако на прошлой неделе был также очень недоволен басаевым и удуговым свидетельствует о том что является владельцем легкового автомобиля мерседес-бенц-600 и дачи площадью 400 кв 
 для попадания в восьмерку сильнейших попал кафельников обыгравший в воскресенье с международного плавучего космодрома морской старт успешно прошел клини

## Задание 2

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf
Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1.	Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

Система, которая использует слова, не входящие в словарь, называется системой с открытым словарем (open vocabulary system). В тестовую выборку добавляется псевдослово UNK. Есть два способа предсказать его вероятность.

1)Превратить систему с открытым словарем в систему с закрытым словарем. Для этого нужно выбрать зафиксированный список слов (словарь). В обучающей выборке каждое несловарное слово конвертировать в токен UNK на этапе нормализации. Оценить вероятность для UNK так же, как для любого другого слова в обучающей выборке. 

2)Если словаря нет, то такой словарь можно создать, заменив слова на токен UNK, основываясь на частотности. Например, мы заменяем на UNK все слова, которые встречаются реже, чем n раз, где n – некоторое небольшое число. Или же задать словарь размером V с запасом (например, 50 тысяч слов), выбрать V самых частотных слов, а остальные заменить на UNK. В любом из этих случаев мы продолжаем обучать языковую модель, как и раньше, рассматривая UNK как обычное слово.

Выбор модели влияет на показатель перплексии.

    2.	Что такое сглаживание (smoothing)?

Слова, которые есть в словаре, могут появиться в тестовой выборке в неизвестном контексте (например, после того слова, после которого они никогда не появлялись при обучении). Чтобы языковая модель не приписывала таким событиям нулевую вероятность, нам нужно сгладить вероятности: повысить вероятности некоторых n-грамм за счет понижения вероятности других. Есть несколько методов сглаживания: сглаживание Лапласа, сглаживание add-k, откат и сглаживание Кнезера-Нея.
